In [ ]:
!pip install inference_sdk

In [ ]:
!git clone https://github.com/nwojke/deep_sort.git
!pip install tensorflow==2.16.1 numpy==1.26.4


In [ ]:
!pip install -U ml_dtypes jax jaxlib

In [ ]:
!pip install numpy==1.26.4 --force-reinstall

In [ ]:
from google.colab import files
uploaded = files.upload()
video_filename = list(uploaded.keys())[0]

In [ ]:
# First, upload the tracker.py file to Colab
from google.colab import files
uploaded = files.upload()

# Then import
from tracker import Tracker

In [ ]:
!mkdir -p model_data
!wget -O model_data/mars-small128.pb https://github.com/Qidian213/deep_sort_yolov3/raw/master/model_data/mars-small128.pb

In [ ]:
from google.colab.patches import cv2_imshow
from inference_sdk import InferenceHTTPClient, InferenceConfiguration
import cv2
import random
from tracker import Tracker

tracker = Tracker()

colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for j in range(10)]
CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="ahSbMeHx177siUJ3OYQr",
)

configuration = InferenceConfiguration(
    confidence_threshold=0.5
)

CLIENT.configure(configuration)

cap = cv2.VideoCapture(video_filename)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    result = CLIENT.infer(frame, model_id="my-first-project-xaurp/4")
    detections = []

    for prediction in result['predictions']:
        x = prediction['x']
        y = prediction['y']
        width = prediction['width']
        height = prediction['height']
        confidence = prediction['confidence']
        class_name = prediction['class']

        # Convert center coordinates to corner coordinates
        x1 = int(x - width/2)
        y1 = int(y - height/2)
        x2 = int(x + width/2)
        y2 = int(y + height/2)

        detections.append([x1, y1, x2, y2, confidence])
    
    tracker.update(frame, detections)     
    
    for track in tracker.tracks:
        bbox = track.bbox
        x1, y1, x2, y2 = bbox
        track_id = track.track_id
        
        # FIXED LINE: Added thickness parameter and converted to integers
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), colors[track_id % len(colors)], 2)
        
        # Optional: Add ID text
        cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1)-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[track_id % len(colors)], 2)
    
    cv2_imshow(frame)

cap.release()